In [13]:
from bs4 import BeautifulSoup
import requests
import time
import re
import os
import csv

In [14]:
# returns a list for apartment, usage to get to new website where we find infomation about particular apartment
def get_apartment_links(apartment_links):
    links = []
    for i in apartment_links:
        text = str(i)
        res =  re.search(r'html', text)
        if res is not None:
            links.append("https://www.athome.de/"+i.attrs['href'])
    return links  

In [15]:
# getting content of all apartments we have links to
def get_content_of_apartment(link_to_apartments):
    store_content = []
    for link_to_apartment in link_to_apartments:
        try:
            get_apartment_information = requests.get(link_to_apartment)
            soup = BeautifulSoup(get_apartment_information.text, 'lxml')
            # get content of characteristics block
            res = soup.find_all('li', {'class': 'feature-bloc-content-specification-content'})
            # print all characteristics from particular apartment
            store_characteristics = []
            for r in res:
                key = r.find('div', {'class': 'feature-bloc-content-specification-content-name'}).text
                value = r.find('div', {'class': 'feature-bloc-content-specification-content-response'}).text
                key = key.replace(" ", "_")
                value = value.replace(" ", "_")
                store_characteristics.append(key +" "+value)
            # store location 
            location = soup.find('div', {'class': 'block-localisation-address'})
            store_characteristics.append("Location "+ location.text.replace(" ", "_"))
            store_content.append(store_characteristics)
        except Exception as e:
            pass
    return store_content

In [16]:
def clean_format(value):
    value = value.replace("_", "")
    return value

In [17]:
def store_output(target_path, target_file, data):
    if not os.path.exists(target_path):
        try:
            os.makedirs(target_path)
        except Exception as e:
            print(e)
            raise

    try:
        sales_price = clean_format(data['Sale_price'])
        livable_surface = clean_format(data['Livable_surface'])
        land = clean_format(data['Land'])
        number_of_rooms = clean_format(data['Number_of_rooms'])
        location = clean_format(data['Location'])
        dict_data = [{'sales_price': f'{sales_price}', 'livable_surface': f'{livable_surface}', 'land': f'{land}', 'number_of_rooms': f'{number_of_rooms}', 'location': f'{location}'}]
    except Exception as e:
        return
    
    csv_columns = ['sales_price','livable_surface','land','number_of_rooms', 'location']
    csv_file = os.path.join(target_path, target_file)+".csv"
    with open(csv_file, 'w', encoding="utf-8") as outfile:
        writer = csv.DictWriter(outfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in dict_data:
            writer.writerow(data)

In [18]:
def make_dic(keys, values):
    data = {}
    for i in range(len(keys)):
        data[keys[i]] = values[i]
    return data

In [19]:
def main(apartment_no, page, initial_page):
    # Pretending to be the browser
    header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36'}

    # Query String Parameters
    payload = {
        'pages': ""
    }
    payload['pages'] = str(page)
    url = f'https://www.athome.de/en/buy/house?pages={initial_page}&page={page}'
    response = requests.get(url, headers=header)
    # Test
    print(response.url)
    # -------
    soup = BeautifulSoup(response.text, 'lxml')
    apartment_links = soup.find_all('a')
    link_to_apartments = get_apartment_links(apartment_links)
    contents = get_content_of_apartment(link_to_apartments)



    # each apartment information is stored in a separate csv file
    # inlcude cases if key is not present
    for content in contents:
        # for one apartment
        keys = []
        values = []
        # making dynamic key value pair, preprocesssing step
        for e in content:
            try:
                key, value = e.split()
                print(e.split())
                keys.append(key)
                values.append(value)
            except Exception as e:
                pass
    
    
        data = make_dic(keys, values)
        store_output('./house_data', f'house{apartment_no}', data)
        apartment_no += 1
        print(f'apartment_no: {apartment_no}')
        

    # after 30 apartments wait for 10 seconds and go on with process
    if(apartment_no % 30 == 0):
        time.sleep(10)
    # wait for 2 seconds    
    time.sleep(2)
    return apartment_no

In [20]:
sites = 75
page = 1 
apartment_no = 1
initial_page = 1
if __name__ == '__main__':
    print("Start scraping...")
    for _ in range(sites):
        apartment_no = main(apartment_no, page, initial_page)
        apartment_no += 1
        page = page + 1
    print("Scraping finished.")

Start scraping...
https://www.athome.de/en/buy/house?pages=1&page=1
['Sale_price', '548,000_€']
['Agency_commission', '2,38_%_inkl._MwSt.']
['Availability', '04/.2022']
['Livable_surface', '123_m²']
['Land', '894_m²']
['Number_of_rooms', '3']
['Number_of_bedrooms', '2']
['Year_of_construction', '2013']
['Renovation_year', '2004']
['Bathroom', '1']
['Restroom', '1']
['Terrace', 'Yes']
['Closed_parking_space', '2']
['Energy_performance_rating_', 'NC']
['Energy_pass_type', 'Energy_performance_pass']
['Energy_pass_expiry_date', '5/05/2023']
['Building_type', 'Not_communicated']
['Location', '_-_Merzkirchen']
apartment_no: 2
['Sale_price', '470,000_€']
['Availability', 'À_convenir']
['Livable_surface', '300_m²']
['Land', '11,458_m²']
['Number_of_rooms', '10']
['Number_of_bedrooms', '6']
['Year_of_construction', '1978']
['Living_room', 'Yes']
['Bathroom', '1']
['Shower_room', '1']
['Basement', 'Yes']
['Fireplace', 'Yes']
['Energy_performance_rating_', 'NC']
['Energy_pass_type', 'NC']
['Build

['Year_of_construction', '2002']
['Renovation_year', '2019']
['Acces_for_mobility-impared_people', 'Yes']
['Living_room', 'Yes']
['Separate_kitchen', 'Yes']
['Open_kitchen', 'Yes']
['Bathooms', '2']
['Shower_rooms', '2']
['Restroom', '1']
['Terrace', '20_m²']
['Garden', '340_m²']
['Open_parking_space', '2']
['Closed_parking_space', '2']
['Basement', 'Yes']
['Laundry', 'Yes']
['Attic', 'Yes']
['Fireplace', 'Yes']
['Parquet', 'Yes']
['Energy_performance_rating_', '33.1A']
['Energy_pass_type', 'Energy_performance_pass']
['Energy_pass_date', '23/10/2020']
['Energy_pass_year', '2020']
['Building_type', 'Residential']
['Pump_heating', 'Yes']
['Geothermal_heating', 'Yes']
apartment_no: 21
https://www.athome.de/en/buy/house?pages=1&page=2
['Sale_price', '355,000_€']
['Availability', 'À_convenir']
['Livable_surface', '183_m²']
['Land', '2,737_m²']
['Number_of_rooms', '7']
['Number_of_bedrooms', '4']
['Year_of_construction', '1900']
['Renovation_year', '2004']
['Living_room', 'Yes']
['Separate_k

https://www.athome.de/en/buy/house?pages=1&page=3
['Sale_price', '329,000_€']
['Availability', 'Immediate']
['Livable_surface', '150_m²']
['Land', '300_m²']
["Property's_floor", '3']
['Total_floors', '3']
['Number_of_rooms', '5']
['Number_of_bedrooms', '2']
['Year_of_construction', '1920']
['Renovated', 'Yes']
['Renovation_year', '2014']
['Living_room', 'Yes']
['Fitted_kitchen', 'Yes']
['Separate_kitchen', 'Yes']
['Bathooms', '2']
['Shower_rooms', '2']
['Terrace', 'Yes']
['Garden', 'Yes']
['Closed_parking_space', '1']
['Basement', 'Yes']
['Laundry', 'Yes']
['Converted_attic', 'Yes']
['Pets_accepted', 'Yes']
['Energy_performance_rating_', '180E']
['Energy_pass_type', 'Energy_consumption_pass']
['Energy_pass_expiry_date', '11/11/2030']
['Energy_pass_date', '12/06/2020']
['Energy_pass_year', '2020']
['Building_type', 'Residential']
['Gas_heating', 'Yes']
['Location', '_1_Kirchenberg_-_Wincheringen']
apartment_no: 44
['Sale_price', '695,000_€']
['Agency_commission', '3_%_zzgl_MwSt.']
['Ava

https://www.athome.de/en/buy/house?pages=1&page=4
['Sale_price', '567,000_€']
['Agency_commission', '3,57_%_inkl._19_%_MwSt.']
['Livable_surface', '180_m²']
['Land', '3,800_m²']
['Number_of_rooms', '7']
['Year_of_construction', '2007']
['Bathooms', '2']
['Terrace', 'Yes']
['Closed_parking_space', '1']
['Energy_performance_rating_', '69.04B']
['Energy_pass_type', 'Energy_performance_pass']
['Energy_pass_expiry_date', '8/02/2030']
['Building_type', 'Not_communicated']
['Gas_heating', 'Yes']
['Location', '_-_Bitburg']
apartment_no: 65
['Sale_price', '579,000_€']
['Agency_commission', '3,57_€']
['Livable_surface', '300_m²']
['Land', '12,000_m²']
['Number_of_rooms', '12']
['Living_room', 'Yes']
['Fitted_kitchen', 'Yes']
['Separate_kitchen', 'Yes']
['Bathooms', '2']
['Terrace', 'Yes']
['Balcony', 'Yes']
['Garden', 'Yes']
['Closed_parking_space', '3']
['Basement', 'Yes']
['Laundry', 'Yes']
['Converted_attic', 'Yes']
['Fireplace', 'Yes']
['Energy_performance_rating_', 'NC']
['Energy_pass_type'

https://www.athome.de/en/buy/house?pages=1&page=6
['Sale_price', '280,000_€']
['Livable_surface', '167_m²']
['Land', '640_m²']
['Number_of_rooms', '4']
['Energy_performance_rating_', 'NC']
['Energy_pass_type', 'NC']
['Building_type', 'Not_communicated']
['Location', '_Riedhofstraße___-_Weißenhorn']
apartment_no: 107
['Sale_price', '390,000_€']
['Livable_surface', '162_m²']
['Land', '585_m²']
['Number_of_rooms', '5']
['Energy_performance_rating_', 'NC']
['Energy_pass_type', 'NC']
['Building_type', 'Not_communicated']
['Location', '_Nadorster_Straße___-_Oldenburg']
apartment_no: 108
['Sale_price', '51,000_€']
['Livable_surface', '99_m²']
['Land', '394_m²']
['Number_of_rooms', '4']
['Energy_performance_rating_', 'NC']
['Energy_pass_type', 'NC']
['Building_type', 'Not_communicated']
['Location', '_Hindenburgstraße___-_Herzberg']
apartment_no: 109
['Sale_price', '123,000_€']
['Livable_surface', '143_m²']
['Land', '662_m²']
['Number_of_rooms', '4']
['Energy_performance_rating_', 'NC']
['Ener

...

['Energy_pass_type', 'NC']
['Building_type', 'Not_communicated']
['Gas_heating', 'Yes']
['Location', '_-_Schweich']
apartment_no: 1386
https://www.athome.de/en/buy/house?pages=1&page=67
['Sale_price', '49,000_€']
['Agency_commission', '0_%']
['Livable_surface', '100_m²']
['Land', '325_m²']
['Number_of_rooms', '6']
['Number_of_bedrooms', '2']
['Year_of_construction', '1964']
['Bathooms', '2']
['Garden', 'Yes']
['Basement', 'Yes']
['Energy_performance_rating_', 'NC']
['Energy_pass_type', 'NC']
['Building_type', 'Not_communicated']
['Location', '_-_Krottelbach']
apartment_no: 1388
['Sale_price', '900,000_€']
['Agency_commission', 'Käufercourtage:_3,48_%_(incl._16_%_MWST)_des_Kaufp']
['Livable_surface', '250_m²']
['Land', '175,400_m²']
['Number_of_rooms', '9']
['Number_of_bedrooms', '6']
['Year_of_construction', '1983']
['Living_room', 'Yes']
['Fitted_kitchen', 'Yes']
['Balcony', 'Yes']
['Garden', 'Yes']
['Closed_parking_space', 'Yes']
['Basement', 'Yes']
['Energy_performance_rating_', 'NC

https://www.athome.de/en/buy/house?pages=1&page=68
['Sale_price', '40,000_€']
['Agency_commission', 'k.A.']
['Livable_surface', '120_m²']
['Land', '493_m²']
['Number_of_rooms', '5']
['Year_of_construction', '1900']
['Renovation_year', '2016']
['Energy_performance_rating_', 'NC']
['Energy_pass_type', 'NC']
['Building_type', 'Not_communicated']
['Location', '_-_Binsfeld']
apartment_no: 1409
['Sale_price', '235,000_€']
['Agency_commission', 'Käufercourtage:_1,78_%_(incl._19%_MWST)_des_Kaufpr']
['Livable_surface', '170_m²']
['Land', '50,000_m²']
['Number_of_rooms', '6']
['Year_of_construction', '1931']
['Renovation_year', '2009']
['Living_room', 'Yes']
['Fitted_kitchen', 'Yes']
['Balcony', 'Yes']
['Garden', 'Yes']
['Basement', 'Yes']
['Energy_performance_rating_', 'E']
['Energy_pass_type', 'Energy_consumption_pass']
['Energy_pass_expiry_date', '28/07/2024']
['Hot_water_included', 'Yes']
['Energy_pass_date', '28/07/2014']
['Energy_pass_year', '2014']
['Building_type', 'Not_communicated']
['

https://www.athome.de/en/buy/house?pages=1&page=69
['Sale_price', '480,000_€']
['Agency_commission', '3,57%_KÃ¤ufercourtage_inkl_MwSt']
['Livable_surface', '600_m²']
['Land', '16,000_m²']
['Number_of_rooms', '25']
['Year_of_construction', '1860']
['Renovation_year', '2009']
['Energy_performance_rating_', 'G']
['Energy_pass_type', 'Energy_performance_pass']
['Building_type', 'Not_communicated']
['Fuel_heating', 'Yes']
['Location', '_-_Börfink']
apartment_no: 1430
['Sale_price', '850,000_€']
['Agency_commission', '3,57_%']
['Livable_surface', '310_m²']
['Land', '15,200_m²']
['Number_of_rooms', '5']
['Number_of_bedrooms', '4']
['Year_of_construction', '1948']
['Renovation_year', '2016']
['Living_room', 'Yes']
['Restroom', '1']
['Energy_performance_rating_', 'NC']
['Energy_pass_type', 'NC']
['Building_type', 'Residential']
['Location', '_Großer_Markt_-_Saarlouis']
apartment_no: 1431
['Sale_price', '298,000_€']
['Agency_commission', '3,57_€']
['Livable_surface', '160_m²']
['Land', '30,000_m

https://www.athome.de/en/buy/house?pages=1&page=70
['Sale_price', '275,696_€']
['Availability', 'To_be_agreed']
['Livable_surface', '142.5_m²']
['Land', '914_m²']
['Number_of_rooms', '4']
['Number_of_bedrooms', '3']
['Renovation_year', '2009']
['Living_room', 'Yes']
['Separate_kitchen', 'Yes']
['Bathroom', '1']
['Restroom', '1']
['Terrace', 'Yes']
['Garden', 'Yes']
['Energy_performance_rating_', 'NC']
['Energy_pass_type', 'NC']
['Building_type', 'Residential']
['Pump_heating', 'Yes']
['Location', '_-_Neuerburg']
apartment_no: 1451
['Sale_price', '480,000_€']
['Agency_commission', 'prov.frei']
['Availability', 'Bezug_nach_Absprache']
['Livable_surface', '404_m²']
['Land', '1,332_m²']
['Number_of_rooms', '10']
['Year_of_construction', '1984']
['Renovation_year', '2019']
['Bathooms', '3']
['Restroom', '1']
['Terrace', 'Yes']
['Balcony', 'Yes']
['Fireplace', 'Yes']
['Parquet', 'Yes']
['Sauna', 'Yes']
['Energy_performance_rating_', 'NC']
['Energy_pass_type', 'NC']
['Building_type', 'Not_com

https://www.athome.de/en/buy/house?pages=1&page=71
['Sale_price', '280,000_€']
['Agency_commission', '3,570%_(inkl._MwSt.)_vom_Kaufpreis']
['Livable_surface', '300_m²']
['Land', '2,400_m²']
['Number_of_rooms', '9']
['Number_of_bedrooms', '4']
['Renovation_year', '2016']
['Bathooms', '2']
['Restroom', '3']
['Terrace', 'Yes']
['Balcony', 'Yes']
['Garden', 'Yes']
['Fireplace', 'Yes']
['Energy_performance_rating_', 'NC']
['Energy_pass_type', 'Energy_performance_pass']
['Building_type', 'Not_communicated']
['Fuel_heating', 'Yes']
['Location', '_-_Wadern']
apartment_no: 1472
['Sale_price', '170,000_€']
['Agency_commission', '3,570%_(inkl._MwSt.)_vom_Kaufpreis']
['Livable_surface', '120_m²']
['Land', '940_m²']
['Number_of_rooms', '4']
['Number_of_bedrooms', '2']
['Renovation_year', '2016']
['Bathroom', '1']
['Restroom', '2']
['Balcony', 'Yes']
['Garden', 'Yes']
['Fireplace', 'Yes']
['Energy_performance_rating_', 'D']
['Energy_pass_type', 'Energy_performance_pass']
['Energy_pass_expiry_date', 